# Aula 04 - [pandas](http://pandas.pydata.org/)

**Objetivos**

- Análise séries temporais
- Ler, manipular e plotar dados tabulares
- Guia de *group-by* e outras operações tabulares avançadas

Ler um `CSV` e mostrar apenas o início da tabela.

In [ ]:
import pandas as pd

pd.read_csv('./data/dados_pirata.csv').head()

O `read_csv` (e vários outros métodos do pandas) possuí uma infinidade de
opções para ajudar na leitura dos dados.

In [ ]:
df = pd.read_csv('./data/dados_pirata.csv',
                 index_col='datahora',
                 na_values=-99999,
                 parse_dates=True).drop('Unnamed: 0', axis=1)

df.tail()

Algumas facilidades do pandas é ter *labels* ricos em informação.  Então vamos renomear as nossas colunas e ordená-las pela profundidade.

In [ ]:
df.columns = ['{0:0>3}'.format(col.split('_')[1]) for col in df.columns]

df.sort(axis=1, inplace=True)

df.head()

Outra facilidade é a limpeza da tabela, aqui jogamos fora todos os inválidos
(*Not-a-Number*) apenas quando todos os valores do eixo 1 (colunas) foram
`NaN`.

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.head()

In [ ]:
desc = df.describe()
desc

Como será a variabilidade desses dados?

In [ ]:
desc.ix['std'] ** 2

In [ ]:
%matplotlib inline

ax = df[['001', '100', '180', '500']].plot()

In [ ]:
from matplotlib import style
import matplotlib.pyplot as plt

style.use('ggplot')

fig, axes = plt.subplots(figsize=(11, 1.25*11), nrows=11, sharex=True, sharey=False)

kw = dict(color='darkorange', linewidth=2)
for ax, col in zip(axes, df.columns):
    ax.plot(df['2015'].index, df['2015'][col], **kw)
    ax.set_ylabel('{} m'.format(col))

Como já havíamos notado antes toda essa série parece ter um *gap* no mesmo 
lugar (provavelmente uma manutenção do fundeio).  Com o pandas podemos 
facilmente interpolar esse *gap* para obtermos uma série contínua.

In [ ]:
df['001'].interpolate().plot()
df['001'].plot()

In [ ]:
df['001'].interpolate(method='time', limit=30).plot()
df['001'].plot()

Com o pandas vocês podem facilmente "re-amostar" uma série ou tabela
para uma nova frequência temporal.  Os dois parâmetros principais para o

In [ ]:
import numpy as np
fig, ax = plt.subplots(figsize=(9, 5))

ax = df.resample('M', how=np.median).plot(ax=ax)

- Group-by months/years (climatology)

In [ ]:
key = lambda x: x.month

grouped = df.groupby(key)

monthly = grouped.mean()
monthly.shape

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

ax = monthly.plot(ax=ax)

In [ ]:
key = lambda x: x.year

grouped = df.groupby(key)

yearly = grouped.mean()
yearly.shape

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

ax = yearly.plot(ax=ax)

In [ ]:
df.resample('A', how='mean').plot()

In [ ]:
for col in df.columns:
    yearly[col] = (yearly[col] - yearly[col].mean()) / yearly[col].std(ddof=0)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))

ax = yearly.plot(ax=ax)

In [ ]:
yearly.corr()

In [ ]:
!head ./data/15t30717.3f1

In [ ]:
from datetime import datetime
from pandas import read_table

cols = ['j', 'u', 'v', 'temp', 'sal', 'y', 'mn', 'd', 'h', 'mi']

df = read_table('./data/15t30717.3f1' , delim_whitespace=True, names=cols)
dates = [datetime(*x) for x in
         zip(df['y'], df['mn'], df['d'], df['h'], df['mi'])]
df.index = dates
df.drop(['y', 'mn', 'd', 'h', 'mi', 'j'], axis=1, inplace=True)
df.head()

In [ ]:
ax = df['v'].plot()

In [ ]:
from oceans import lanc

freq = 1./40  # Hours
window_size = 96+1+96
pad = np.zeros(window_size) * np.NaN

wt = lanc(window_size, freq)
res = np.convolve(wt, df['v'], mode='same')

df['low'] = res
df['high'] = df['v'] - df['low']

In [ ]:
ax = df[['low', 'high']].plot()

https://ocefpaf.github.io/python4oceanographers/blog/2013/07/29/python-ctd/